In [2]:
!pip install lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 53.0 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [4]:
import os
import torch
from torch import nn
import torch.nn.functional as F
import torchvision
from torchvision import models, datasets, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from torchmetrics.functional import accuracy


In [5]:
class DataModule(pl.LightningDataModule):
  def __init__(self, batch_size, data_dir: str = '/content/drive/MyDrive/dogcat/chomeo'):
    super().__init__()
    self.batch_size = batch_size
    self.data_dir = data_dir
    self.transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
    ])
    self.dims = (3,224,224)
    self.num_classes = 2

  def prepare_data(self):
    train_path = os.path.join(self.data_dir,'train', 'train')
    test_path = os.path.join(self.data_dir, 'test', 'test')

    self.dataset_train = torchvision.datasets.ImageFolder(train_path, transform = self.transform)
    self.dataset_test = torchvision.datasets.ImageFolder(test_path, transform= self.transform)

  def setup(self, stage =None):
    if stage == 'fit' or stage is None:
      dataset_full = torchvision.datasets.ImageFolder(self.data_dir, transform = self.transform)
      dataset_size = len(dataset_full)

      train_size = int(dataset_size * 0.8)
      val_size =dataset_size - train_size

      self.dataset_train, self.dataset_val = random_split(dataset_full,[train_size, val_size])

    if stage == 'test' or stage is None:
      self.dataset_test = torchvision.datasets.ImageFolder(self.data_dir, transform = self.transform)

  def train_dataloader(self):
    return DataLoader(self.dataset_train, batch_size = self.batch_size, shuffle = True)

  def val_dataloader(self):
    return DataLoader(self.dataset_val, batch_size  =self.batch_size)

  def test_dataloader(self):
    return DataLoader(self.dataset_test, batch_size = self.batch_size)




In [6]:
class Model(pl.LightningModule):
  def __init__(self, num_classes,learning_rate = 0.0001):
    super().__init__()

    #log parameters
    self.save_hyperparameters()
    self.learning_rate = learning_rate
    self.num_classes = num_classes
    #model
    self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64,64,kernel_size = 3, padding = 1 ),
            nn.ReLU(inplace = True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(512,512 ,kernel_size=3, padding=1),
            nn.ReLU(inplace = True),
            nn.Conv2d(512,512, kernel_size=3, padding=1),
            nn.ReLU(inplace = True),
            nn.Conv2d(512,512, kernel_size=3, padding=1),
            nn.ReLU(inplace = True),
            nn.MaxPool2d(kernel_size=2, stride=2),

        )
    self.avgpool = nn.AdaptiveAvgPool2d((7, 7))

    self.classifier = nn.Sequential(
           nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p = 0.5),
            nn.Linear(4096,2048),
            nn.ReLU(inplace=True),
            nn.Dropout(p = 0.5),
            nn.Linear(2048,1024 ),
            nn.ReLU(inplace=True),
            nn.Dropout(p = 0.5),
            nn.Linear(1024, num_classes),

    )

  def forward(self, x):
    x = self.features(x)
    x = self.avgpool(x)
    x = torch.flatten(x,1)
    # x = x.view(x.size(0), -1)  # Flatten
    x = self.classifier(x)
    return x

  def training_step(self, batch, batch_idx):
    x, y = batch
    logits = self(x)
    # probabilities = F.softmax(logits, dim = 1)
    loss = F.cross_entropy(logits, y)
    preds = torch.argmax(logits, dim = 1)
    acc = accuracy(preds, y ,task = 'MULTICLASS', num_classes = self.num_classes)
    self.log('train_loss', loss, on_step=True, on_epoch=True, logger=True)
    self.log('train_acc', acc, on_step=True, on_epoch=True, logger=True)

    return loss

  def validation_step(self, batch, batch_idx):
    x, y = batch
    logits = self(x)
    # probabilities = F.softmax(logits, dim = 1)
    loss = F.cross_entropy(logits, y)
    preds = torch.argmax(logits, dim = 1)
    acc = accuracy(preds, y, task = 'MULTICLASS',num_classes = self.num_classes)
    self.log('val_loss', loss, on_step=True, on_epoch=True, logger=True)
    self.log('val_acc', acc, on_step=True, on_epoch=True, logger=True)

    return loss

  def test_step(self, batch, batch_idx):
    x, y = batch
    logits = self(x)
    # probabilities = F.softmax(logits, dim = 1)
    loss = F.cross_entropy(logits, y)
    preds = torch.argmax(logits, dim = 1)
    acc = accuracy(preds, y, task = 'MULTICLASS',num_classes = self.num_classes)
    self.log('test_loss', loss, on_step=True, on_epoch=True, logger=True)
    self.log('test_acc', acc, on_step=True, on_epoch=True, logger=True)

    return loss

  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr = self.learning_rate)
    return optimizer



In [7]:
dm = DataModule(batch_size = 32)
dm.prepare_data()
dm.setup()
model = Model(dm.num_classes)

checkpoint_callback = pl.callbacks.ModelCheckpoint()
class LogCallback(pl.Callback):
    def on_train_epoch_end(self, trainer, pl_module, outputs= None):
        train_loss = trainer.callback_metrics.get('train_loss_epoch', 'N/A')
        train_acc = trainer.callback_metrics.get('train_acc_epoch', 'N/A')
        print(f'Epoch {trainer.current_epoch} - Train Loss: {train_loss}, Train Acc: {train_acc}')

    def on_validation_epoch_end(self, trainer, pl_module):
        val_loss = trainer.callback_metrics.get('val_loss_epoch', 'N/A')
        val_acc = trainer.callback_metrics.get('val_acc_epoch', 'N/A')
        print(f'Epoch {trainer.current_epoch} - Validation Loss: {val_loss}, Validation Acc: {val_acc}')

    def on_test_epoch_end(self, trainer, pl_module):
        test_loss = trainer.callback_metrics.get('test_loss_epoch', 'N/A')
        test_acc = trainer.callback_metrics.get('test_acc_epoch', 'N/A')
        print(f'Epoch {trainer.current_epoch} - Test Loss: {test_loss}, Test Acc: {test_acc}')

trainer = pl.Trainer(max_epochs=5,
                     callbacks=[checkpoint_callback, LogCallback()]  # Thêm callback LogCallback
                    )

trainer.fit(model, dm)
trainer.test(dataloaders=dm.test_dataloader())



INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type              | Params
-------------------------------------------------
0 | features   | Sequential        | 9.4 M 
1 | avgpool    | AdaptiveAvgPool2d | 0     
2 | classifier | Sequential        | 113 M 
-------------------------------------------------
122 M     Trainable params
0         Non-trainable params
122 M     Total params
490.642   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.


Epoch 0 - Validation Loss: 0.6867512464523315, Validation Acc: 0.90625


/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.


...

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:145: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/lightning_logs/version_0/checkpoints/epoch=4-step=1755.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at /content/lightning_logs/version_0/checkpoints/epoch=4-step=1755.ckpt
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have

Testing: |          | 0/? [00:00<?, ?it/s]

Epoch 5 - Test Loss: 0.9645126461982727, Test Acc: 0.5002471804618835


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      test_acc_epoch       │    0.5002471804618835     │
│      test_loss_epoch      │    0.9645126461982727     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss_epoch': 0.9645126461982727, 'test_acc_epoch': 0.5002471804618835}]